In [2]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.models import Sequential

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2 as IRV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception

In [3]:
train_dir="../input/inaturalist/inaturalist_12K/train/"
test_dir="../input/inaturalist/inaturalist_12K/val/"

In [4]:
!pip install wandb
import wandb
wandb.login()

In [5]:
# configure sweep parameters
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  "parameters": {
                   'first_layer_filters': {'values': [32, 64]},
                   'filter_org': {'values': [0.5, 1, 2]}, # Halving, same, doubling in subsequent layers
                   'data_aug': {'values': [False, True]},
                   'batch_norm': {'values': [False, True]}, 
                   'dropout': {'values': [0.0, 0.2, 0.3]},
                   'kernel_size': {'values': [3, 5, 7]},
                   'dense_size': {'values': [64,512,1024]},
                   'activation': {'values': ['relu']},
                   'num_epochs': {'values': [10,20,30]}, 
                   'optimizer': {'values': ['adam']},
                   'conv_layers': {'values': [5]}
                }
}

sweep_id = wandb.sweep(sweep_config,project='CS6910 Assignment2', entity='go4rav')

In [ ]:




sweep_config = {'name': 'Bayesian Sweep', 'method': 'bayes'}
sweep_config['metric'] = {'name': 'val_acc', 'goal': 'maximize'}
parameters_dict = {
                   'first_layer_filters': {'values': [32, 64]},
                   'filter_org': {'values': [0.5, 1, 2]}, # Halving, same, doubling in subsequent layers
                   'data_aug': {'values': [False, True]},
                   'batch_norm': {'values': [False, True]}, 
                   'dropout': {'values': [0.0, 0.2, 0.3]},
                   'kernel_size': {'values': [3, 5, 7]},
                   'dense_size': {'values': [64,512,1024]},
                   'activation': {'values': ['relu']},
                   'num_epochs': {'values': [10,20,30]}, 
                   'optimizer': {'values': ['adam']},
                   'conv_layers': {'values': [5]}
                  }
sweep_id = wandb.sweep(sweep_config,project='CS6910 Assignment2', entity='go4rav')

In [ ]:
img_height,img_width=(256,256)

augment= ImageDataGenerator(
rotation_range=30,
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
rescale=1.0 / 255,
validation_split=0.1,
)

datagen = ImageDataGenerator(
rescale=1.0 / 255,
validation_split=0.1,
)

train_set = datagen.flow_from_directory(
train_dir,
target_size=(img_height, img_width),
batch_size=32,
color_mode='rgb',
class_mode='sparse',
shuffle=True,
subset='training',
seed=123,
)

train = augment.flow_from_directory(
  train_dir,
  target_size=(img_height, img_width),
  batch_size=32,
  color_mode='rgb',
  class_mode='sparse',
  shuffle=True,
  subset='training',
  seed=123,
)

validate = datagen.flow_from_directory(
  train_dir,
  target_size=(img_height, img_width),
  color_mode='rgb',
  class_mode='sparse',
  shuffle=True,
  subset='validation',
  seed=123,
)

train = np.asarray(train)
validate = np.asarray(validate)

In [ ]:
dataset_augment = ImageDataGenerator(rescale=1. / 255)
train = dataset_augment.flow_from_directory(train_dir,
                                            shuffle=True, target_size=(256, 256), batch_size=32)
validate = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
    test_dir, shuffle=False, target_size=(256,256))

In [6]:
def train():
    wandb.init()
    CONFIG=wandb.config
    filters = CONFIG.first_layer_filters
    dropout_rate = CONFIG.dropout
    kernel_size = CONFIG.kernel_size
    dense_neurons = CONFIG.dense_size
    epochs = CONFIG.num_epochs
    input_shape = (256,256,3)
    with tf.device('/device:GPU:0'):
      # tf.keras.backend.clear_session()
      model = Sequential() 
      for i in range(0,5): 
          model.add(Conv2D(filters, kernel_size= (3,3), input_shape=input_shape, activation='relu'))
          model.add(MaxPooling2D(pool_size=(2,2))) 
          #model.add(BatchNormalization())
            
      model.add(Flatten()) 
      model.add(Dense(dense_neurons, activation='relu'))
      model.add(tf.keras.layers.Dropout(rate=dropout_rate))
      model.add(Dense(10, activation='softmax'))
        
      img_height,img_width=(256,256)

      augment= ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      rescale=1.0 / 255,
      validation_split=0.1,
       )

      datagen = ImageDataGenerator(
      rescale=1.0 / 255,
      validation_split=0.1,
      )

      train_set = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
      )

      train = augment.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size=32,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
      )

      validate = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='validation',
      seed=123,
      )

      optimiser = tf.keras.optimizers.Adam(learning_rate=.0004)
      model.compile(optimizer=optimiser, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
      model.fit(train, epochs=epochs, validation_data=validate)
  # model.add(Dense(64, activation=tf.nn.relu)) 
  # # model.add(Dropout(0.2)) 
  # model.add(Dense(10,activation = tf.nn.softmax))  


In [ ]:
wandb.agent('go4rav/CS6910 Assignment2/ihiq3hik', train ,count = 100)
